In [ ]:
train_folder = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/train'
validation_folder = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/validation'
test_folder = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/test'

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import save_model

def load_and_process_file(file_path):
    data = np.load(file_path)
    img = data[..., 4]  # Getting the fifth slice across the third dimension
    min_val = np.min(img)
    max_val = np.max(img)
    normalized_data = ((img - min_val) / (max_val - min_val)) # Normalizing the data to be in range [0, 1]
    # sobel_x = cv2.Sobel(normalized_data, cv2.CV_64F, 1, 0)
    # sobel_y = cv2.Sobel(normalized_data, cv2.CV_64F, 0, 1)
    # sobel_mag = np.sqrt(np.square(sobel_x) + np.square(sobel_y))
    # clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    # clahe_image = clahe.apply(normalized_data)
    return normalized_data




In [ ]:
import os
import random
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, base_dir, batch_size=32):
        self.base_dir = base_dir
        self.batch_size = batch_size

        # List all directories in the base directory
        self.image_dirs = [os.path.join(base_dir, x) for x in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, x))]
        self.indices = np.arange(len(self.image_dirs))

    def __len__(self):
        return len(self.image_dirs) // self.batch_size

    def __getitem__(self, index):
        # Select self.batch_size directories
        batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch_dirs = [self.image_dirs[i] for i in batch_indices]

        # Prepare empty arrays for our batch
        images = np.empty((self.batch_size, 256, 256, 3))
        labels = np.empty((self.batch_size, 256, 256, 1))

        for i, dir_path in enumerate(batch_dirs):
            # Load each of the three bands, process it, and stack them
            band_08 = load_and_process_file(os.path.join(dir_path, 'band_08.npy'))
            band_12 = load_and_process_file(os.path.join(dir_path, 'band_12.npy'))
            band_16 = load_and_process_file(os.path.join(dir_path, 'band_16.npy'))
            images[i] = np.stack([band_08, band_12, band_16], axis=-1)

            # Load the label and add it to our labels array
            labels[i] = np.load(os.path.join(dir_path, 'human_pixel_masks.npy'))

        return images, labels


In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dropout
import tensorflow as tf

# source
# https://github.com/nikhilroxtomar/Semantic-Segmentation-Architecture/blob/main/TensorFlow/efficientnetb0_unet.py

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

def build_effienet_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained Encoder """
    encoder = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=inputs)

    s1 = encoder.get_layer("input_1").output                      ## 256
    s2 = encoder.get_layer("block2a_expand_activation").output    ## 128
    s3 = encoder.get_layer("block3a_expand_activation").output    ## 64
    s4 = encoder.get_layer("block4a_expand_activation").output    ## 32

    """ Bottleneck """
    b1 = encoder.get_layer("block6a_expand_activation").output    ## 16

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                               ## 32
    d2 = decoder_block(d1, s3, 256)                               ## 64
    d3 = decoder_block(d2, s2, 128)                               ## 128
    d4 = decoder_block(d3, s1, 64)                                ## 256

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="EfficientNetB0_UNET")
    return model



In [ ]:
import keras.backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

input_shape = (256, 256, 3)
model = build_effienet_unet(input_shape)

train_gen = DataGenerator(train_folder)
val_gen = DataGenerator(validation_folder)

# Compile the model
model.compile(optimizer=Adam(), loss=dice_coef_loss, metrics=[jacard_coef, dice_coef])

# Define the filepath format for saving the model weights
checkpoint_path = "/kaggle/working/weights.h5"

# Define a custom callback to print validation metrics
class ValidationMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs['val_loss']
        val_jacard_coef = logs['val_jacard_coef']
        val_dice_coef = logs['val_dice_coef']
        print(f'\nEpoch {epoch + 1}: val_loss = {val_loss:.4f}, val_jacard_coef = {val_jacard_coef:.4f}, val_dice_coef = {val_dice_coef:.4f}\n')

# Create an instance of the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

# Train the model with the checkpoint and custom validation metrics callbacks
history = model.fit(train_gen, validation_data=val_gen, epochs=3, steps_per_epoch=312,
          validation_steps=58, callbacks=[checkpoint_callback, ValidationMetricsCallback()])


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get the next batch of validation data
images, true_masks = next(iter(val_gen))

# Predict masks on the images
predicted_masks = model.predict(images)

# Rescale the masks back to 0-255 range
# predicted_masks = (predicted_masks * 255).astype(np.uint8)

# Iterate over the images, true masks, and predicted masks and plot them
for i in range(len(images)):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    # Plot image
    axs[0].imshow(images[i])
    axs[0].set_title('Image')

    # Plot true mask
    axs[1].imshow(true_masks[i].squeeze(), cmap='gray')
    axs[1].set_title('True Mask')

    # Plot predicted mask
    axs[2].imshow(predicted_masks[i].squeeze(), cmap='gray')
    axs[2].set_title('Predicted Mask')

    # Hide axes
    axs[0].axis('off')
    axs[1].axis('off')
    axs[2].axis('off')

    plt.show()
